# Web scraping using selenium
## Group Members

| No. | Name             |
|-----|------------------|
| 1   | JOSEPH LAU YEO KAI A22EC0055         |
| 2   | VINESH A/L VIJAYA KUMAR A22EC0290   |
| 3   | VINESH A/L VIJAYA KUMAR A22EC0290    |
| 4   | NUR FARAH ADIBAH BINTI IDRIS A22EC0245       |




In [ ]:
# Install necessary Python packages
!pip install selenium webdriver_manager chromedriver_autoinstaller

# Install required Linux packages
!apt-get update -y
!apt-get install -y wget unzip

# Download Chrome and ChromeDriver for headless use in Colab
!wget -q https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.96/linux64/chrome-linux64.zip
!wget -q https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.96/linux64/chromedriver-linux64.zip

# Extract the downloaded zip files
!unzip chrome-linux64.zip
!unzip chromedriver-linux64.zip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 20.4 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,659 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphic

In [ ]:
!uname -a

Linux 0aadc46e54af 6.1.123+ #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [ ]:
# Set the system PATH to include Chrome and ChromeDriver binaries
import os
os.environ['PATH'] += f":{os.getcwd()}/chrome-linux64:{os.getcwd()}/chromedriver-linux64"

# limit page
|   | Category                   | URL                                                                                                                          |
| - | -------------------------- | ---------------------------------------------------------------------------------------------------------------------------- |
| 0 | NST Viral                  | [https://www.nst.com.my/news/nst-viral?page=81](https://www.nst.com.my/news/nst-viral?page=81)                               |
| 1 | Crime & Courts             | [https://www.nst.com.my/news/crime-courts?page=1518](https://www.nst.com.my/news/crime-courts?page=1518)                     |
| 2 | Nation                     | [https://www.nst.com.my/news/nation?page=8994](https://www.nst.com.my/news/nation?page=8994)                                 |
| 3 | Government & Public Policy | [https://www.nst.com.my/news/government-public-policy?page=56](https://www.nst.com.my/news/government-public-policy?page=56) |
| 4 | Politics                   | [https://www.nst.com.my/news/politics?page=1032](https://www.nst.com.my/news/politics?page=1032)                             |



# News Category


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

from urllib.parse import urljoin

import time
import csv
import os
import sys


In [ ]:

# Configuration
BASE_URL = 'https://www.nst.com.my'
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver_path = os.path.join(os.getcwd(), 'chromedriver-linux64', 'chromedriver')
    service = Service(executable_path=driver_path)
    return webdriver.Chrome(service=service, options=chrome_options)


def scrape_nst_articles(url, driver):
    articles_data = []

    try:
        print(f"Accessing URL: {url}")
        driver.get(url)

        # Wait for articles to load
        print("Waiting for content to load...")
        WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "article-teaser"))
        )

        articles = driver.find_elements(By.CLASS_NAME, "article-teaser")
        print(f"Found {len(articles)} articles")

        for i, article in enumerate(articles, 1):
            try:
                headline_element = article.find_element(By.CLASS_NAME, "field-title")
                headline = headline_element.text.strip()

                # Extract Summary
                try:
                    summary_element = article.find_element(By.CLASS_NAME, "article-teaser")
                    summary = summary_element.text.strip()
                except:
                    summary = ""

                # Extract Section and Date
                try:
                    meta_div = driver.find_element(By.CLASS_NAME, "article-meta")
                    # Get section
                    try:
                        section_element = meta_div.find_element(By.CLASS_NAME, "field-category")
                        section = section_element.text.strip()
                    except:
                        section = "Unknown"

                    # Get date
                    try:
                        date_element = meta_div.find_element(By.CLASS_NAME, "created-ago")
                        date = date_element.text.strip()
                    except:
                        date = "Unknown"

                except:
                    section = "Unknown"
                    date = "Unknown"

                if headline and date:
                    articles_data.append({
                        'Section': section,
                        'Date': date,
                        'Headline': headline,
                        'Summary': summary,
                    })

            except Exception as e:

                continue

    except Exception as e:
        print(f"Error accessing website: {e}")

    return articles_data



def save_to_csv(articles_data, filename):
    if not articles_data:
        print("No articles to save.")
        return

    fieldnames = ['Section', 'Date', 'Headline', 'Summary']


    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(articles_data)
        print(f"Saved {len(articles_data)} articles to {filename}")

def main():

    news_cat = ['nation']

    all_articles = []

    try:
        # Setup WebDriver once for all pages
        driver = setup_driver()
        for cat in news_cat:
            news_path = f'/news/{cat}'
            full_path = urljoin(BASE_URL, news_path)

            for page in range(1,1320):
                print(f"\nScraping page {page} for {cat.capitalize()}...")
                page_url = f"{full_path}?page={page}"
                articles_data = scrape_nst_articles(page_url, driver)
                if not articles_data:
                    print(f"No articles found on page {page} or error accessing page. Stopping pagination for {cat}.")
                    continue
                all_articles.extend(articles_data)
                print(f"Successfully scraped {len(articles_data)} articles from page {page}")
                time.sleep(0.5)


        filename = f'nst_articles_nation_page_1_1300.csv'
        save_to_csv(all_articles, filename)

        if all_articles:
            print(f"\nTotal articles scraped: {len(all_articles)}")
            print(f"Articles saved to: {filename}")
        else:
            print(f"\nNo articles were found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if driver:
            print("Closing browser...")
            driver.quit()

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 487

Scraping page 488 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=488
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 488

Scraping page 489 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=489
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 489

Scraping page 490 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=490
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 490

Scraping page 491 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=491
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 491

Scraping page 492 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=492
Waiting f

In [ ]:

# Configuration
BASE_URL = 'https://www.nst.com.my'

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver_path = os.path.join(os.getcwd(), 'chromedriver-linux64', 'chromedriver')
    service = Service(executable_path=driver_path)
    return webdriver.Chrome(service=service, options=chrome_options)


def scrape_nst_articles(url, driver):
    articles_data = []

    try:
        print(f"Accessing URL: {url}")
        driver.get(url)

        # Wait for articles to load
        print("Waiting for content to load...")
        WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "article-teaser"))
        )

        articles = driver.find_elements(By.CLASS_NAME, "article-teaser")
        print(f"Found {len(articles)} articles")

        for i, article in enumerate(articles, 1):
            try:
                headline_element = article.find_element(By.CLASS_NAME, "field-title")
                headline = headline_element.text.strip()

                # Extract Summary
                try:
                    summary_element = article.find_element(By.CLASS_NAME, "article-teaser")
                    summary = summary_element.text.strip()
                except:
                    summary = ""

                # Extract Section and Date
                try:
                    meta_div = driver.find_element(By.CLASS_NAME, "article-meta")
                    # Get section
                    try:
                        section_element = meta_div.find_element(By.CLASS_NAME, "field-category")
                        section = section_element.text.strip()
                    except:
                        section = "Unknown"

                    # Get date
                    try:
                        date_element = meta_div.find_element(By.CLASS_NAME, "created-ago")
                        date = date_element.text.strip()
                    except:
                        date = "Unknown"

                except:
                    section = "Unknown"
                    date = "Unknown"

                if headline and date:
                    articles_data.append({
                        'Section': section,
                        'Date': date,
                        'Headline': headline,
                        'Summary': summary,
                    })

            except Exception as e:

                continue

    except Exception as e:
        print(f"Error accessing website: {e}")

    return articles_data



def save_to_csv(articles_data, filename):
    if not articles_data:
        print("No articles to save.")
        return

    fieldnames = ['Section', 'Date', 'Headline', 'Summary']


    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(articles_data)
        print(f"Saved {len(articles_data)} articles to {filename}")

def main():

    news_cat = ['nation']

    all_articles = []

    try:
        # Setup WebDriver once for all pages
        driver = setup_driver()
        for cat in news_cat:
            news_path = f'/news/{cat}'
            full_path = urljoin(BASE_URL, news_path)

            for page in range(2751,4000):
                print(f"\nScraping page {page} for {cat.capitalize()}...")
                page_url = f"{full_path}?page={page}"
                articles_data = scrape_nst_articles(page_url, driver)
                if not articles_data:
                    print(f"No articles found on page {page} or error accessing page. Stopping pagination for {cat}.")
                    continue
                all_articles.extend(articles_data)
                print(f"Successfully scraped {len(articles_data)} articles from page {page}")
                time.sleep(0.5)


        filename = f'nst_articles_nation_page_2751_4000.csv'
        save_to_csv(all_articles, filename)

        if all_articles:
            print(f"\nTotal articles scraped: {len(all_articles)}")
            print(f"Articles saved to: {filename}")
        else:
            print(f"\nNo articles were found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if driver:
            print("Closing browser...")
            driver.quit()

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 3167

Scraping page 3168 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=3168
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 3168

Scraping page 3169 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=3169
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 3169

Scraping page 3170 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=3170
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 3170

Scraping page 3171 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page=3171
Waiting for content to load...
Found 40 articles
Successfully scraped 20 articles from page 3171

Scraping page 3172 for Nation...
Accessing URL: https://www.nst.com.my/news/nation?page